In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-05-08 20:51:57--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.07s   

2023-05-08 20:51:58 (15.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# Tokenizer
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('hello world'))
print(decode(encode('hello world')))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [6]:
# Tokenize data
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:100])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [7]:
# split data
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'{context=} {target=}')

context=tensor([18]) target=tensor(47)
context=tensor([18, 47]) target=tensor(56)
context=tensor([18, 47, 56]) target=tensor(57)
context=tensor([18, 47, 56, 57]) target=tensor(58)
context=tensor([18, 47, 56, 57, 58]) target=tensor(1)
context=tensor([18, 47, 56, 57, 58,  1]) target=tensor(15)
context=tensor([18, 47, 56, 57, 58,  1, 15]) target=tensor(47)
context=tensor([18, 47, 56, 57, 58,  1, 15, 47]) target=tensor(58)


In [10]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size + 1] for i in ix])
    return x, y
    
xb, yb = get_batch('train')
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)


for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'{context=} {target=}')

torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
context=tensor([24]) target=tensor(43)
context=tensor([24, 43]) target=tensor(58)
context=tensor([24, 43, 58]) target=tensor(5)
context=tensor([24, 43, 58,  5]) target=tensor(57)
context=tensor([24, 43, 58,  5, 57]) target=tensor(1)
context=tensor([24, 43, 58,  5, 57,  1]) target=tensor(46)
context=tensor([24, 43, 58,  5, 57,  1, 46]) target=tensor(43)
context=tensor([24, 43, 58,  5, 57,  1, 46, 43]) target=tensor(39)
context=tensor([44]) target=tensor(53)
context=tensor([44, 53]) target=tensor(56)
context=tensor([44, 53, 56]) target=tensor(1)
context=tensor([44, 53, 56,  1]) target=tensor(58)
context=tensor([44, 53,

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx)
            logits = logits[:, -1,:]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim =1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(f'{loss}')
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

NameError: name 'out' is not defined

In [12]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [19]:
batch_size = 32

eval_interval = 300
eval_iters = 200

@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

for iter in range(10000):
    
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step: {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    xb, yb = get_batch('train')
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0: train loss 2.4626, val loss 2.4903
step: 300: train loss 2.4506, val loss 2.4852
step: 600: train loss 2.4577, val loss 2.4905
step: 900: train loss 2.4562, val loss 2.4975
step: 1200: train loss 2.4534, val loss 2.4908
step: 1500: train loss 2.4578, val loss 2.4914
step: 1800: train loss 2.4565, val loss 2.4842
step: 2100: train loss 2.4351, val loss 2.4788
step: 2400: train loss 2.4418, val loss 2.5007
step: 2700: train loss 2.4533, val loss 2.4826
step: 3000: train loss 2.4533, val loss 2.4875
step: 3300: train loss 2.4453, val loss 2.4853
step: 3600: train loss 2.4449, val loss 2.4877
step: 3900: train loss 2.4517, val loss 2.4913
step: 4200: train loss 2.4505, val loss 2.4855
step: 4500: train loss 2.4552, val loss 2.4873
step: 4800: train loss 2.4516, val loss 2.4858
step: 5100: train loss 2.4512, val loss 2.4888
step: 5400: train loss 2.4516, val loss 2.4845
step: 5700: train loss 2.4492, val loss 2.4901
step: 6000: train loss 2.4521, val loss 2.4767
step: 6300: train l

In [14]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))



Figsmy m wowe cand y, tourou thnothins, tht fo meat:
So o an Thamyo ptithee thoud te.
BYotherquat n


In [22]:
torch.manual_seed(1337)
B, T, C = 4,8,32
x = torch.randn(B, T, C)
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, C)
wei = q @ k.transpose(-2, -1) # (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
#out = wei @ x
v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 32])